# Text vectorization

In [9]:
# imports

from gensim.models import Word2Vec
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [10]:
# word2vec model

# Load text
data = pd.read_csv('News_Category_Dataset_IS_course_preprocessed.csv')

# Train test split
X_train, X_test, y_train, y_test = train_test_split(
    data[['headline', 'clean_headline', 'short_description', 'clean_short_description']], 
    data['category'], 
    test_size=0.2, 
    random_state=42)

# Tokenize text
X_train['combined_text'] = X_train['headline'].str.cat(X_train['short_description'], sep=' ')
X_test['combined_text'] = X_test['headline'].str.cat(X_test['short_description'], sep=' ')

tokenized_train_text = [str(text).split() for text in X_train['combined_text']]
tokenized_test_text = [str(text).split() for text in X_test['combined_text']]

# Train word2vec model
w2v_model = Word2Vec(tokenized_train_text, vector_size=100, window=5, min_count=1, workers=4, epochs=10)

In [11]:
# Test a few different classifiers with Word2Vec

# Function to get average Word2Vec representation for a sentence
def get_average_w2v(tokens):
    vector_sum = 0
    count = 0
    for word in tokens:
        if word in w2v_model.wv:
            vector_sum += w2v_model.wv[word]
            count += 1
    if count != 0:
        return vector_sum / count
    else:
        return [0] * 100  # Return zero vector if no word found

# Add Word2Vec representations to DataFrame
X_train_w2v = [get_average_w2v(text) for text in tokenized_train_text]
X_test_w2v = [get_average_w2v(text) for text in tokenized_test_text]

# Logistic Regression model
logistic_model = LogisticRegression()
logistic_model.fit(X_train_w2v, y_train)
logistic_predictions = logistic_model.predict(X_test_w2v)
logistic_accuracy = accuracy_score(y_test, logistic_predictions)
print("Logistic Regression Accuracy:", logistic_accuracy)

# Random Forest model
rf_model = RandomForestClassifier()
rf_model.fit(X_train_w2v, y_train)
rf_predictions = rf_model.predict(X_test_w2v)
rf_accuracy = accuracy_score(y_test, rf_predictions)
print("Random Forest Accuracy:", rf_accuracy)

c:\Users\Domen\IS\Inteligentni-sistemi\Inteligentni-sistemi\.venv\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression Accuracy: 0.5459426603181036
Random Forest Accuracy: 0.49387093506230373
